In [1]:
import numpy as np
import scipy.optimize as opt
import scipy.stats as sts
import matplotlib.pyplot as plt
from scipy.integrate import quad

## Problem 4.

In [2]:
c, k, w, r = np.loadtxt(r'C:\Users\Max\Documents\BootCamp2017\Econ\Wk4_StrEst\data\MacroSeries.txt', delimiter=',').T

In [69]:
def get_z(alpha, r, k):

    zi = np.log(r) - np.log(alpha) - (alpha - 1) * np.log(k)
    
    return zi

def get_moms(params, c, k, w, r):
    alpha, beta, rho, mu = params
    
    zi = get_z(alpha, r, k)
    z_t = zi[:-1]
    z_t1 = zi[1:] 
    
    mom_1 = np.mean(z_t1 - rho * z_t - (1 - rho) * mu)
    mom_2 = np.mean((z_t1 - rho * z_t - (1 - rho) * mu) * z_t)
    
    
    mom_3 = np.mean(beta * alpha * np.exp(z_t1) * k[1:] ** (alpha - 1) *
                     (c[:-1] / c[1:])-1)
    
    mom_4 = np.mean((beta * alpha * np.exp(z_t1) * k[1:] ** (alpha - 1) *
                     (c[:-1] / c[1:])-1) * w[:-1])
    
#     mom_3 = np.mean(np.log(beta) + alpha + zi[1:] + (alpha - 1) * k[1:] +
#                    c[:-1] - c[1:])
#     mom_4 = np.mean((np.log(beta) + alpha  + zi[1:] + (alpha - 1) * k[1:] +
#                    c[:-1] - c[1:]) * w[:-1])
    return np.array([mom_1, mom_2, mom_3, mom_4])

def criterion(params, *args):
    alpha, beta, rho, mu = params
    c, k, w, r, W = args
    
    mom_args = c, k, w, r
    err_vec = get_moms(params, c, k, w, r)
    
    crit_val = np.dot(np.dot(err_vec.T, W), err_vec)
    
    return crit_val
    

In [71]:
alpha_0 = 0.0000001
beta_0 = 0.01
rho_0 = 0.8
mu_0 = 10
params_init = np.array([alpha_0, beta_0, rho_0, mu_0])
W = np.identity(4)
data = (c, k, w, r, W)

In [72]:
get_z(alpha_0, r, k)
get_moms(params_init, c, k, w, r)
criterion(params_init, c, k, w, r, W)

106301274989438.58

In [73]:
res = opt.minimize(criterion, params_init, args=data,
                   method='L-BFGS-B',
                   bounds=((1e-10, 1-1e-10), (1e-10, 1-1e-10), 
                          (1e-10 - 1, 1-1e-10), (1e-10, None)))
res

      fun: 0.0010119580209931851
 hess_inv: <4x4 LbfgsInvHessProduct with dtype=float64>
      jac: array([  6.03766904e-03,   4.55503474e+05,  -2.60832758e-02,
         0.00000000e+00])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 235
      nit: 34
   status: 0
  success: True
        x: array([ 0.98443373,  0.99      ,  1.        ,  9.9958868 ])

In [74]:
alpha, beta, rho, mu = res.x
crit = res.fun

print('Estimation results',
      '\n\nalpha:\t\t\t', alpha,
      '\nbeta:\t\t\t', beta,
      '\nrho:\t\t\t', rho,
      '\nmu :\t\t\t', mu,
     '\ncriterion function:\t', crit)
print()

Estimation results 

alpha:			 0.984433730729 
beta:			 0.989999997058 
rho:			 0.9999999999 
mu :			 9.99588679925 
criterion function:	 0.00101195802099



Beta is going close to 0.99 regardless of the initial values. However, all othher values only change very little compared to their strating values. This does not seem very reliable. Moreover, the criterion function changes only in the 5th decimals when I plug in substattially different values.